In [7]:
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob


In [8]:
football_feed_df = pd.read_pickle('football_feed_df.pkl')
print(football_feed_df.columns)
football_feed_df.head()

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'extended_entities', 'metadata',
       'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'team',
       'user_id', 'retweeted_status_id', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'text', 'hashtags',
       'user_mentions', 'emoji', 'processed_text'],
      dtype='object')


,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,metadata,source,...,user_id,retweeted_status_id,quoted_status_id,quoted_status_id_str,quoted_status,text,hashtags,user_mentions,emoji,processed_text
0,Fri Nov 13 21:27:20 +0000 2020,1327362434361823232,1327362434361823232,- Heisman Trophy winner (1956)\n- No. 1 overal...,False,"[0, 212]","{'hashtags': [{'text': 'Packers', 'indices': [...","{'media': [{'id': 1327340427352174592, 'id_str...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://studio.twitter.com"" rel=""nofo...",...,35865630,NaN,NaN,NaN,NaN,NaN,"[{'text': 'Packers', 'indices': [182, 190]}]",,[],- Heisman Trophy winner ( 1956 ) - No . 1 over...
1,Fri Nov 13 19:36:20 +0000 2020,1327334499919642625,1327334499919642625,"Paul Hornung, described by Vince Lombardi as ""...",False,"[0, 151]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,35865630,NaN,NaN,NaN,NaN,NaN,[],,[],"Paul Hornung , described Vince Lombardi `` gre..."
2,Fri Nov 13 18:30:00 +0000 2020,1327317809945325569,1327317809945325569,The entire Pro Football HOF family mourns the ...,False,"[0, 264]","{'hashtags': [{'text': 'HOFForever', 'indices'...","{'media': [{'id': 1327312635180834816, 'id_str...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://ads.twitter.com"" rel=""nofollo...",...,23632910,NaN,NaN,NaN,NaN,NaN,"[{'text': 'HOFForever', 'indices': [242, 253]}]",Green Bay Packers,[],The entire Pro Football HOF family mourns pass...
3,Sat Nov 14 20:07:06 +0000 2020,1327704633796653064,1327704633796653064,"To all the true heroes, we say thank you. \n\...",False,"[0, 112]","{'hashtags': [{'text': 'SaluteToService', 'ind...","{'media': [{'id': 1326259298645790722, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,724773710549372928,1.327628e+18,NaN,NaN,NaN,NaN,"[{'text': 'SaluteToService', 'indices': [57, 7...",Green Bay Packers,[],"To true hero , say thank . # SaluteToService |"
4,Sat Nov 14 20:06:46 +0000 2020,1327704548304150529,1327704548304150529,Domino’s delivery guy to my house.,False,"[13, 47]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,1314216459892133888,NaN,NaN,NaN,NaN,NaN,[],PFF GB Packers,[],Domino ’ delivery guy house .


In [9]:
#check to see how mnay repeat tweets we scraped. 

#i am unsure how to handle this 

print(len(football_feed_df.full_text))
print(len(set(football_feed_df.full_text)))

1200
894


In [10]:
#shuffle the data 
np.random.seed(1)
df = football_feed_df.reindex(np.random.permutation(football_feed_df.index))

In [11]:
#data is already preprocessed in scrape script

def sentiment_analysis(text):
    text = " ".join(text["processed_text"])
    text = TextBlob(text)

    return text.sentiment.polarity  #returns float in range [-1, 1]

teams_df = football_feed_df[["processed_text", "team"]].groupby("team")
sentiments = teams_df.apply(sentiment_analysis)

In [12]:
sentiments

team
Bears      0.096870
Chiefs     0.062429
Cowboys    0.176234
Eagles     0.245052
Packers    0.274715
Raiders    0.102080
dtype: float64